# WRF Downloader Notebook

### Imports

In [6]:
import xarray as xr
import rioxarray
import pandas as pd
import numpy as np
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os
import xcdat as xc
from metpy.calc import relative_humidity_from_specific_humidity
from metpy.units import units
from datetime import datetime as dt

### Download files from S3

Open Dataset on AWS S3 `s3://wrf-cmip6-noversioning/downscaled_products/gcm`. 

From there, lots of different runs, bias corrected runs suffixed `_bc`. 

Can see directory to browse here: https://wrf-cmip6-noversioning.s3.amazonaws.com/index.html#downscaled_products/gcm/

In [10]:
from concurrent.futures import ThreadPoolExecutor

BUCKET_NAME = 'wrf-cmip6-noversioning'
OUTPUT_DIR = 'data/'
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED, max_pool_connections=24))

def generateFileNames(start_date: str, end_date: str, model: str, data_tier: int, domain: int, historical: bool, bias_correction: bool) -> list[str]:
    r = pd.date_range(start_date, end_date, freq='1h', inclusive='left', normalize=True)
    file_prefix = {1: "wrfout", 2: "auxhist"}
    path = f'downscaled_products/gcm/{model}{"_historical" if historical else ""}{"_bc" if bias_correction else ""}/hourly'
    # Gross check since files start sept 1 in each yearly directory
    return ["%s/%s/d0%s/%s_d01_%s" % (path, d.year if d.month > 9 else d.year - 1, domain, file_prefix[data_tier], pd.to_datetime(d).strftime('%Y-%m-%d_%H:%M:%S')) for d in r]

def downloadS3File(bucket: str, file: str, output_dir: str) -> str:
    if output_dir[-1] == '/':
        output_dir = output_dir[:-1]
    output_file = "%s/%s_%s.nc" % (output_dir, file.split('/')[2], file.split('/')[-1].replace(':', '-'))
    s3.download_file(bucket, file, output_file)
    return output_file

files = generateFileNames('2020-01-01', '2020-01-08', 'cesm2_r11i1p1f1_ssp245', 2, 2, False, False)

start_time = dt.now()
with ThreadPoolExecutor(24) as executor:
    downloaded_files = executor.map(lambda file: downloadS3File(BUCKET_NAME, file, OUTPUT_DIR), files)

end_time = dt.now()

print('Time to download {} files: {} seconds'.format(len(list(downloaded_files)), (end_time - start_time).seconds))


Time to download 168 files: 30 seconds


In [9]:
x = xr.open_mfdataset("data/*.nc", combine='nested', concat_dim='Time')
x

<xarray.Dataset> Size: 1GB
Dimensions:  (Time: 168, south_north: 340, west_east: 270)
Dimensions without coordinates: Time, south_north, west_east
Data variables: (12/22)
    Times    (Time) |S19 3kB dask.array<chunksize=(1,), meta=np.ndarray>
    Q2       (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    T2       (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    PSFC     (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    U10      (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    V10      (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    ...       ...
    LWUPB    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    LWUPBC   (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    LWDNB    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    LWDNBC   (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    RUNSF    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    RUNSB    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
Attributes: (12/85)
    TITLE:                            OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE:                      2019-12-19_00:00:00
    WEST-EAST_GRID_DIMENSION:        271
    SOUTH-NORTH_GRID_DIMENSION:      341
    BOTTOM-TOP_GRID_DIMENSION:       40
    DX:                              9000.0
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

### Clip to Polygon, Calculate Windspeed and Relative Humidity and Total Precip

In [10]:
# Calculate Relative Humidity using Specific Humidity, Temperature, and Pressure 
# See https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.relative_humidity_from_specific_humidity.html
x = x.assign(RH=relative_humidity_from_specific_humidity(x.PSFC * units.Pa, x.T2 * units.degK , x.Q2))
# Calculate Windspeed (directionless) using U and V Components
x = x.assign(WS10=np.sqrt(x.U10**2 + x.V10**2))
x = x.assign(TP=x.RAINC + x.RAINNC)

# Keep Just the Vars we want
x = x[['T2', 'RH', 'WS10', 'TP', 'LWDNB', 'SWDNB']]

In [11]:
x

<xarray.Dataset> Size: 370MB
Dimensions:  (Time: 168, south_north: 340, west_east: 270)
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    T2       (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    RH       (Time, south_north, west_east) float32 62MB <Quantity(dask.array...
    WS10     (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    TP       (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    LWDNB    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
    SWDNB    (Time, south_north, west_east) float32 62MB dask.array<chunksize=(1, 170, 135), meta=np.ndarray>
Attributes: (12/85)
    TITLE:                            OUTPUT FROM WRF V4.1.3 MODEL
    START_DATE:                      2019-12-19_00:00:00
    WEST-EAST_GRID_DIMENSION:        271
    SOUTH-NORTH_GRID_DIMENSION:      341
    BOTTOM-TOP_GRID_DIMENSION:       40
    DX:                              9000.0
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      0
    ETAC:                            0.0

In [125]:
import geojson
with open('skagit_boundaries.json') as f:
    c = geojson.load(f)
    x.rio.clip(c)
    print(c)

MissingCRS: CRS not found. Please set the CRS with 'rio.write_crs()'. Data variable: Q2

In [ ]:
# Parse GeoJson polygon - see skagit_boundaries.json for example
def parseGeoJson(geojson_path: str) -> tuple[float, float, float, float]:
    with open(geojson_path) as f:
        poly = geojson.load(f)
        bounds = list(geojson.utils.coords(poly))
        long, lat = zip(*bounds)
        return (min(long), max(long), min(lat), max(lat))

def limitGeographicRange(bounds: tuple[float, float, float, float], subsetFiles: list) -> list:
    return [wgrib2.region(f, bounds, name='skagit-basin') for f in subsetFiles]
    
bounds = parseGeoJson('skagit_boundaries.json')
geo_limited_files = limitGeographicRange(bounds, fh_files)

def calc_windspeed()
# ws = sqrt(u^2 + v^2)
# metpy to convert spefici humidity to rh by passing in temp and pressure
# metpy.calc.relative_humidity_from_specific_humidity

#Tier 2 vars - temp, specific humidity, u & v wind components (get speed), Surface downwelled solar radiation,Surface downwelled longwave radiation, precip - combine convective and non-convective